In [1]:
# Data libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load datasets
eurostoxx_original = pd.read_csv("eurostoxx.csv", low_memory=False)
eurostoxx_isin = pd.read_csv("eurostoxx_ISINs.csv")
trucost = pd.read_csv("07062024_Cushon_Trucost_Data.csv")
isin_summary = pd.read_csv("isin_summary.csv", low_memory=False)
index_price = pd.read_csv("EuroSTOXX600 Historical Price Data.csv")

In [3]:
# Data cleaning
# for eurostoxx_original
eurostoxx_original.columns = ['Date'] + list(eurostoxx_original.columns[1:]) #rename the columns
eurostoxx_original = eurostoxx_original.drop(index=0) #drop the first row
eurostoxx_original = eurostoxx_original.melt(id_vars=['Date'],var_name='Ticker',value_name='Price') #transformation

# for isin_summary
isin_summary.columns = ['ISIN'] + list(isin_summary.columns[1:]) #rename the columns

# Ensure the Date column is in datetime format
eurostoxx_original['Date'] = pd.to_datetime(eurostoxx_original['Date'])
index_price['Date'] = pd.to_datetime(index_price['Date'])

/tmp/ipykernel_311/4281198837.py:12: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  index_price['Date'] = pd.to_datetime(index_price['Date'])


In [4]:
# Merge eurostoxx_original and eurostoxx_isin
eurostoxx = pd.merge(eurostoxx_original, eurostoxx_isin, on='Ticker', how='inner').drop(columns='Ticker')
# Create a 'Year' column in eurostoxx for merging with 'FISCAL_YEAR'
eurostoxx['Year'] = eurostoxx['Date'].dt.year

In [5]:
# Find the sector for each company in eurostoxx
eurostoxx_sectors = eurostoxx_isin.drop(columns=['Ticker']).merge(isin_summary[['ISIN','sector','industry']], 
                                                                  on='ISIN', how='left')
# Fill the nan
sectors_nan = eurostoxx_sectors[eurostoxx_sectors['sector'].isna()]

sectors_nan

,ISIN,Name,sector,industry
0,NL0006294274,Euronext NV,NaN,NaN
1,CH0018294154,PSP Swiss Property AG,NaN,NaN
2,SE0020050417,Boliden AB,NaN,NaN
3,FR0013227113,SOITEC,NaN,NaN
8,DE0006452907,Nemetschek SE,NaN,NaN
...,...,...,...,...
591,DE0006766504,Aurubis AG,NaN,NaN
594,CH0466642201,Helvetia Holding AG,NaN,NaN
595,CH0319416936,Flughafen Zurich AG,NaN,NaN
596,CH0239229302,SFS Group AG,NaN,NaN


In [6]:
eurostoxx_sectors['sector'].dropna().unique()

array(['Real Estate', 'Basic Materials', 'Industrials',
       'Communication Services', 'Consumer Defensive', 'Healthcare',
       'Financial Services', 'Energy', 'Utilities', 'Technology',
       'Consumer Cyclical'], dtype=object)

In [7]:
# Filter companies in scope of EU ETS
sectors_in_scope = ['Basic Materials', 'Energy', 'Utilities']
filtered_companies = eurostoxx_sectors[eurostoxx_sectors['sector'].isin(sectors_in_scope)]['ISIN']
filtered_companies

5      AT0000831706
11     GB0007188757
15     GB00B1XZS820
17     PLPKN0000018
22     ES0130670112
           ...     
556    CH0012214059
560    FI0009005987
584    GB00B1FH8J72
592    SE0011090018
597    JE00B4T3BW64
Name: ISIN, Length: 68, dtype: object

In [8]:
# Merge data
d = pd.merge(eurostoxx, trucost, left_on=['ISIN', 'Year'], right_on=['ISIN', 'FISCAL_YEAR'], how='inner')
d = d.merge(index_price[['Date', 'Change %']], on='Date', how='left').drop(columns=['FISCAL_YEAR','COMPANY_NAME'])
d

,Date,Price,ISIN,Name,Year,CARBON_SCOPE_1,CARBON_INTENSITY_SCOPE_1_USD,CARBON_SCOPE_2,CARBON_INTENSITY_SCOPE_2_USD,CARBON_SCOPE_3_UPSTREAM,CARBON_SCOPE_3_UPSTREAM_INTENSITY_USD,CARBON_SCOPE_3_DOWNSTREAM,CARBON_INTENSITY_SCOPE_3_DOWNSTREAM_USD,Change %
0,2017-01-02,35.581,NL0006294274,Euronext NV,2017,316.093049,0.526847,4957.096043,8.26223,18918.29512,31.532032,7.801486e+00,0.013003,0.86%
1,2017-01-03,38.06,NL0006294274,Euronext NV,2017,316.093049,0.526847,4957.096043,8.26223,18918.29512,31.532032,7.801486e+00,0.013003,1.47%
2,2017-01-04,37.918,NL0006294274,Euronext NV,2017,316.093049,0.526847,4957.096043,8.26223,18918.29512,31.532032,7.801486e+00,0.013003,NaN
3,2017-01-05,38.041,NL0006294274,Euronext NV,2017,316.093049,0.526847,4957.096043,8.26223,18918.29512,31.532032,7.801486e+00,0.013003,-0.12%
4,2017-01-06,37.58,NL0006294274,Euronext NV,2017,316.093049,0.526847,4957.096043,8.26223,18918.29512,31.532032,7.801486e+00,0.013003,0.43%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890488,2022-12-26,85.7,PLPEKAO00016,Bank Polska Kasa Opieki SA,2022,8687.000000,4.111000,38762.000000,18.34500,41887.27700,19.824000,2.271414e+06,1074.996948,NaN
890489,2022-12-27,84.86,PLPEKAO00016,Bank Polska Kasa Opieki SA,2022,8687.000000,4.111000,38762.000000,18.34500,41887.27700,19.824000,2.271414e+06,1074.996948,0.13%
890490,2022-12-28,85.66,PLPEKAO00016,Bank Polska Kasa Opieki SA,2022,8687.000000,4.111000,38762.000000,18.34500,41887.27700,19.824000,2.271414e+06,1074.996948,-0.13%
890491,2022-12-29,87.4,PLPEKAO00016,Bank Polska Kasa Opieki SA,2022,8687.000000,4.111000,38762.000000,18.34500,41887.27700,19.824000,2.271414e+06,1074.996948,0.68%


In [9]:
# Count the number of rows with missing values
d.isna().any(axis=1).sum()

120946

In [10]:
# Drop rows with missing values
d = d.dropna()

In [11]:
# Ensure all relevant columns are numeric
cols_to_check = d.columns.drop(['Date','ISIN','Name','Change %'])
for col in cols_to_check:
    d[col] = pd.to_numeric(d[col], errors='coerce')
    
d['Change %'] = d['Change %'].str.replace('%', '').astype(float)

/tmp/ipykernel_311/976622525.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[col] = pd.to_numeric(d[col], errors='coerce')
/tmp/ipykernel_311/976622525.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['Change %'] = d['Change %'].str.replace('%', '').astype(float)


In [12]:
# Filter for in and out of scope companies
d_in = d[d['ISIN'].isin(filtered_companies)]
d_out = d[~d['ISIN'].isin(filtered_companies)]

d_in

,Date,Price,ISIN,Name,Year,CARBON_SCOPE_1,CARBON_INTENSITY_SCOPE_1_USD,CARBON_SCOPE_2,CARBON_INTENSITY_SCOPE_2_USD,CARBON_SCOPE_3_UPSTREAM,CARBON_SCOPE_3_UPSTREAM_INTENSITY_USD,CARBON_SCOPE_3_DOWNSTREAM,CARBON_INTENSITY_SCOPE_3_DOWNSTREAM_USD,Change %
7825,2017-01-02,16.845,AT0000831706,Wienerberger AG,2017,1.960073e+06,537.902,4.826224e+05,132.446,6.486585e+05,178.011,1.977635e+06,542.721613,0.86
7826,2017-01-03,17.120,AT0000831706,Wienerberger AG,2017,1.960073e+06,537.902,4.826224e+05,132.446,6.486585e+05,178.011,1.977635e+06,542.721613,1.47
7828,2017-01-05,17.005,AT0000831706,Wienerberger AG,2017,1.960073e+06,537.902,4.826224e+05,132.446,6.486585e+05,178.011,1.977635e+06,542.721613,-0.12
7829,2017-01-06,17.005,AT0000831706,Wienerberger AG,2017,1.960073e+06,537.902,4.826224e+05,132.446,6.486585e+05,178.011,1.977635e+06,542.721613,0.43
7830,2017-01-09,16.965,AT0000831706,Wienerberger AG,2017,1.960073e+06,537.902,4.826224e+05,132.446,6.486585e+05,178.011,1.977635e+06,542.721613,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888922,2022-12-23,558.000,JE00B4T3BW64,Glencore PLC,2022,1.660700e+07,64.875,1.043400e+07,40.760,1.877569e+07,73.347,3.767529e+08,1471.782837,0.04
888924,2022-12-27,558.000,JE00B4T3BW64,Glencore PLC,2022,1.660700e+07,64.875,1.043400e+07,40.760,1.877569e+07,73.347,3.767529e+08,1471.782837,0.13
888925,2022-12-28,559.700,JE00B4T3BW64,Glencore PLC,2022,1.660700e+07,64.875,1.043400e+07,40.760,1.877569e+07,73.347,3.767529e+08,1471.782837,-0.13
888926,2022-12-29,558.300,JE00B4T3BW64,Glencore PLC,2022,1.660700e+07,64.875,1.043400e+07,40.760,1.877569e+07,73.347,3.767529e+08,1471.782837,0.68


In [13]:
# Prepare the data for regression
X = d_in[['CARBON_INTENSITY_SCOPE_1_USD', 'CARBON_INTENSITY_SCOPE_2_USD', 
          'CARBON_SCOPE_3_UPSTREAM_INTENSITY_USD', 'CARBON_INTENSITY_SCOPE_3_DOWNSTREAM_USD', 'Change %']]
y = d_in.groupby('ISIN')['Price'].pct_change().rename('Return')

In [14]:
# Drop the first row from both X and y to handle NaN in y
X = X.iloc[1:].copy()
y = y.iloc[1:].copy()

# Remove infinite values
X = X[np.isfinite(X).all(1)]
y = y[np.isfinite(y)]

# Align indices after removing infinite values
common_index = X.index.intersection(y.index)
X = X.loc[common_index]
y = y.loc[common_index]

# Add a constant term for intercept
X = sm.add_constant(X)

In [15]:
# Regression
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Return   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     1959.
Date:                Wed, 12 Jun 2024   Prob (F-statistic):               0.00
Time:                        23:01:20   Log-Likelihood:             2.3044e+05
No. Observations:               90287   AIC:                        -4.609e+05
Df Residuals:                   90281   BIC:                        -4.608e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                       0.0003      0.000      2.636      0.008    8.61e-05       0.001
CARBON_INTENSITY_SCOPE_1_USD            -1.446e-08    6.8e-08     -0.213      0.832   -1.48e-07    1.19e-07
CARBON_INTENSITY_SCOPE_2_USD             3.247e-07   4.71e-07      0.690      0.490   -5.98e-07    1.25e-06
CARBON_SCOPE_3_UPSTREAM_INTENSITY_USD   -3.807e-07   3.98e-07     -0.955      0.339   -1.16e-06       4e-07
CARBON_INTENSITY_SCOPE_3_DOWNSTREAM_USD  2.596e-08    2.1e-08      1.237      0.216   -1.52e-08    6.71e-08
Change %                                    0.0060   6.05e-05     98.942      0.000       0.006       0.006
==============================================================================
Omnibus:                    16562.131   Durbin-Watson:                   1.951
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           364729.872
Skew:                          -0.268   Prob(JB):                         0.00
Kurtosis:                      12.832   Cond. No.                     6.78e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Define pipelines
Lin_Reg = Pipeline([
    ('Linear_Regression', LinearRegression())
])

Poly_Reg_2 = Pipeline([
    ('Polynomial_Features', PolynomialFeatures(degree=2)),
    ('Linear_Regression', LinearRegression())
])

Poly_Reg_3 = Pipeline([
    ('Polynomial_Features', PolynomialFeatures(degree=3)),
    ('Linear_Regression', LinearRegression())
])

RidgeCV_Reg = Pipeline([
    ('RidgeCV', RidgeCV(alphas=np.arange(0.01, 2.01, 0.01), cv=5))
])

LassoCV_Reg = Pipeline([
    ('LassoCV', LassoCV(cv=5, random_state=42))
])

# Fit the models
Lin_Reg.fit(X, y)
Poly_Reg_2.fit(X, y)
Poly_Reg_3.fit(X, y)
RidgeCV_Reg.fit(X, y)
LassoCV_Reg.fit(X, y)

# Predict and evaluate the models
models = {
    'Linear Regression': Lin_Reg,
    'Polynomial Regression Degree 2': Poly_Reg_2,
    'Polynomial Regression Degree 3': Poly_Reg_3,
    'Ridge Regression': RidgeCV_Reg,
    'Lasso Regression': LassoCV_Reg
}

for name, model in models.items():
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    print(f'{name} Performance:')
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r2}\n')

Linear Regression Performance:
Mean Squared Error: 0.0003553248675669915
R-squared: 0.09785597539012503

Polynomial Regression Degree 2 Performance:
Mean Squared Error: 0.0003511386840738051
R-squared: 0.10848439115555575

Polynomial Regression Degree 3 Performance:
Mean Squared Error: 0.0003492120207369283
R-squared: 0.11337604939692736

Ridge Regression Performance:
Mean Squared Error: 0.000355324867583331
R-squared: 0.09785597534864021

Lasso Regression Performance:
Mean Squared Error: 0.0003553870149906163
R-squared: 0.0976981875965155

